# Analyze eBPF Programs

This notebook compiles, analyzes, and generates the report for eBPF programs. 

The raw report is saved in the `output/report` directory.

In [1]:
%reload_ext autoreload
%autoreload 2

The following cell compiles the BCC tools. 

In [2]:
from utils import BCC_TOOLS_PATH

!make -C {BCC_TOOLS_PATH} -j $(nproc)

make: Entering directory '/Users/szhong/Code/DepSurf/software/bcc/libbpf-tools'
make: Nothing to be done for 'all'.
make: Leaving directory '/Users/szhong/Code/DepSurf/software/bcc/libbpf-tools'


The following cell compiles the eBPF program for Tracee.

In [3]:
from utils import TRACEE_PATH

!make -C {TRACEE_PATH} bpf

make: Nothing to be done for 'bpf'.


The following cell analyzes and generates the report for eBPF programs.

In [2]:
from depsurf import BPFProgram, VersionGroup, DepReport, DepKind
from utils import TRACEE_PATH, BCC_OBJ_PATH, OUTPUT_PATH, save_pkl

REPORT_PATH = OUTPUT_PATH / "report"
REPORT_PATH.mkdir(parents=True, exist_ok=True)
GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH]
PROG_PATHS = sorted(BCC_OBJ_PATH.glob("*.bpf.o")) + [
    TRACEE_PATH / "dist" / "tracee.bpf.o"
]


REPORT_KINDS = [
    DepKind.FUNC,
    DepKind.TRACEPOINT,
    DepKind.LSM,
    DepKind.FIELD,
    DepKind.STRUCT,
    DepKind.SYSCALL,
]

results = {}
for path in PROG_PATHS:
    bpf = BPFProgram(path)
    result_path = REPORT_PATH / f"{bpf.name}.md"
    with open(result_path, "w") as f:
        prog_report = {}
        for dep in bpf.deps:
            if dep.kind not in REPORT_KINDS:
                print(f"{dep}: Skipped", file=f)
                continue
            report = DepReport.from_groups(dep, GROUPS)
            report.print(file=f)
            prog_report[dep] = report
        print(f"Report saved to {result_path}", flush=True)
    results[bpf.name] = prog_report


save_pkl(results, "bcc")

Report saved to /Users/szhong/Code/DepSurf/output/report/bashreadline.md
Report saved to /Users/szhong/Code/DepSurf/output/report/bindsnoop.md
Report saved to /Users/szhong/Code/DepSurf/output/report/biolatency.md
Report saved to /Users/szhong/Code/DepSurf/output/report/biopattern.md
Report saved to /Users/szhong/Code/DepSurf/output/report/biosnoop.md
Report saved to /Users/szhong/Code/DepSurf/output/report/biostacks.md
Report saved to /Users/szhong/Code/DepSurf/output/report/biotop.md
Report saved to /Users/szhong/Code/DepSurf/output/report/bitesize.md
Report saved to /Users/szhong/Code/DepSurf/output/report/cachestat.md
Report saved to /Users/szhong/Code/DepSurf/output/report/capable.md
Report saved to /Users/szhong/Code/DepSurf/output/report/cpudist.md
Report saved to /Users/szhong/Code/DepSurf/output/report/cpufreq.md
Report saved to /Users/szhong/Code/DepSurf/output/report/drsnoop.md
Report saved to /Users/szhong/Code/DepSurf/output/report/execsnoop.md
Report saved to /Users/szhon

[ utils_pickle.py:13 ] INFO: Saved bcc to /Users/szhong/Code/DepSurf/output/bcc.pkl
